In [1]:
# Load packages
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import numpy as np
import pandas as pd
import os
import scipy as scp
import scipy.stats as scps
import time
from datetime import datetime


# Load my own functions
import dnnregressor_train_eval_keras as dnnk
from kde_training_utilities import kde_load_data
from kde_training_utilities import kde_make_train_test_split
import make_data_wfpt as mdw

In [2]:
# Handle some cuda business

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16348381029164850821
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 7930876733673738056
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 12048649421
locality {
  bus_id: 1
  links {
  }
}
incarnation: 72182277514369685
physical_device_desc: "device: 0, name: GeForce GTX TITAN X, pci bus id: 0000:03:00.0, compute capability: 5.2"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 12856655918892481973
physical_device_desc: "device: XLA_GPU device"
]


In [3]:
# Make dnnk class (cpm for choice probability model)
cpm = dnnk.dnn_trainer()

# Define folder in which dataset lies
data_folder = '/media/data_cifs/afengler/data/kde/full_ddm/train_test_data/'

In [4]:
# Make train test split
kde_make_train_test_split(folder = data_folder,
                          p_train = 0.8)

get files in folder
check if we have a train and test sets already


'looks like a train test split exists in folder: Please remove before running this function'

In [5]:
# Load train test split
cpm.data['train_features'], cpm.data['train_labels'], cpm.data['test_features'], cpm.data['test_labels'] = kde_load_data(folder = data_folder)

In [6]:
cpm.data['test_features'].shape

(111566113, 7)

In [7]:
cpm.data['train_features'].shape

(446278279, 7)

In [ ]:
cpm.data['train_features'].iloc[171247010, :]

In [ ]:
cpm.data['train_features']['log_l'] = cpm.data['train_labels']

In [ ]:
cpm.data['train_features'].sort_values(by = 'log_l')

In [ ]:
cpm.data['train_features']

In [ ]:
cpm.data['train_features'].iloc[22428, :]

In [ ]:
cpm.data['train_labels'][22428, ]

In [8]:
# Make all parameters we can specify explicit
# Model parameters
cpm.model_params

{'input_shape': 3,
 'output_shape': 1,
 'output_activation': 'sigmoid',
 'hidden_layers': [20, 20, 20, 20],
 'hidden_activations': ['relu', 'relu', 'relu', 'relu'],
 'l1_activation': [0.0, 0.0, 0.0, 0.0],
 'l2_activation': [0.0, 0.0, 0.0, 0.0],
 'l1_kernel': [0.0, 0.0, 0.0, 0.0],
 'l2_kernel': [0.0, 0.0, 0.0, 0.0],
 'optimizer': 'Nadam',
 'loss': 'mse',
 'metrics': ['mse']}

In [9]:
# Parameters governing training
cpm.train_params

{'callback_funs': ['ReduceLROnPlateau', 'EarlyStopping', 'ModelCheckpoint'],
 'plateau_patience': 10,
 'min_delta': 0.0001,
 'early_stopping_patience': 15,
 'callback_monitor': 'loss',
 'min_learning_rate': 1e-07,
 'red_coef_learning_rate': 0.1,
 'ckpt_period': 10,
 'ckpt_save_best_only': True,
 'ckpt_save_weights_only': True,
 'max_train_epochs': 2000,
 'batch_size': 10000,
 'warm_start': False,
 'checkpoint': 'ckpt',
 'model_cnt': 0}

In [10]:
# Parameters concerning data storage
cpm.data_params

{'data_type': 'choice_probabilities',
 'model_directory': '/media/data_cifs/afengler/git_repos/nn_likelihoods/keras_models',
 'checkpoint': 'ckpt',
 'model_name': 'dnnregressor',
 'data_type_signature': '_choice_probabilities_analytic_',
 'timestamp': '07_02_19_18_18_12',
 'training_data_size': 2500000}

In [11]:
# If necessary, specify new set of parameters here:
# Model params
cpm.model_params['output_activation'] = 'linear'
cpm.model_params['hidden_layers'] = [20, 40, 60, 80, 100, 120]
cpm.model_params['hidden_activations'] = ['relu', 'relu', 'relu', 'relu', 'relu', 'relu']
cpm.model_params['input_shape'] = cpm.data['train_features'].shape[1]
# cpm.model_params['l1_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# cpm.model_params['l2_activation'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l1_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
cpm.model_params['l2_kernel'] = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# Train params
cpm.train_params['batch_size'] = 1000000
cpm.train_params['max_train_epochs'] = 250
cpm.train_params['min_delta'] = 0.00001


# Data params
cpm.data_params['data_type'] = 'kde'
cpm.data_params['data_type_signature'] = '_full_ddm_'
cpm.data_params['training_data_size'] = cpm.data['train_features'].shape[0]
cpm.data_params['timestamp'] = datetime.now().strftime('%m_%d_%y_%H_%M_%S')
cpm.data_params['model_directory'] = '/media/data_cifs/afengler/data/kde/full_ddm/keras_models/'

In [12]:
# Make model
cpm.keras_model_generate(save_model = True)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [13]:
# Train model
cpm.run_training(save_history = True, 
                 warm_start = False)

Train on 446278279 samples, validate on 111566113 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/250
446278279/446278279 [==============================] - 1209s 3us/sample - loss: 0.0186 - mean_squared_error: 0.0186 - val_loss: 0.0045 - val_mean_squared_error: 0.0045
Epoch 2/250
446278279/446278279 [==============================] - 1531s 3us/sample - loss: 0.0032 - mean_squared_error: 0.0032 - val_loss: 0.0047 - val_mean_squared_error: 0.0047
Epoch 3/250
446278279/446278279 [==============================] - 1236s 3us/sample - loss: 0.0022 - mean_squared_error: 0.0022 - val_loss: 0.0018 - val_mean_squared_error: 0.0018
Epoch 4/250
446278279/446278279 [==============================] - 1418s 3us/sample - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0013 - val_mean_squared_error: 0.0013
Epoch 5/250
446278279/446278279 [==============================] - 1419s 3us/sample - loss: 0.0014 - mean_squared_error: 0.0014 - val_loss: 0.0012 - val_mean_squared_error: 0

Epoch 32/250
446278279/446278279 [==============================] - 1330s 3us/sample - loss: 2.9696e-04 - mean_squared_error: 2.9696e-04 - val_loss: 2.9511e-04 - val_mean_squared_error: 2.9511e-04
Epoch 33/250
446278279/446278279 [==============================] - 1354s 3us/sample - loss: 2.9401e-04 - mean_squared_error: 2.9401e-04 - val_loss: 2.9292e-04 - val_mean_squared_error: 2.9292e-04
Epoch 34/250
446278279/446278279 [==============================] - 1392s 3us/sample - loss: 2.9176e-04 - mean_squared_error: 2.9176e-04 - val_loss: 2.9062e-04 - val_mean_squared_error: 2.9062e-04
Epoch 35/250
446278279/446278279 [==============================] - 1410s 3us/sample - loss: 2.8938e-04 - mean_squared_error: 2.8938e-04 - val_loss: 2.8811e-04 - val_mean_squared_error: 2.8811e-04
Epoch 36/250
446278279/446278279 [==============================] - 1428s 3us/sample - loss: 2.8673e-04 - mean_squared_error: 2.8673e-04 - val_loss: 2.8531e-04 - val_mean_squared_error: 2.8531e-04
Epoch 37/250
44

,loss,mean_squared_error,val_loss,val_mean_squared_error,lr
0,0.018647,0.018647,0.004492,0.004492,0.002000
1,0.003249,0.003249,0.004684,0.004684,0.002000
2,0.002198,0.002198,0.001775,0.001775,0.002000
3,0.001688,0.001688,0.001304,0.001304,0.002000
4,0.001450,0.001450,0.001174,0.001174,0.002000
5,0.001216,0.001216,0.001538,0.001538,0.002000
6,0.001109,0.001109,0.000672,0.000672,0.002000
7,0.000989,0.000989,0.000887,0.000887,0.002000
8,0.000918,0.000918,0.001224,0.001224,0.002000
9,0.000828,0.000828,0.000729,0.000729,0.002000
